In [76]:
from infrastructure.database.mongo_client import MongoDBClient
from infrastructure.repository import YoutubeContentRepository
from use_case import YouTubeScriptRefinement
from domain.youtube_timeline_summary import YoutubeTimelineSummary
from domain.youtube_timeline_section import YoutubeTimelineSection
from domain.youtube_content import YouTubeContent


import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

# MongoDB 클라이언트 초기화
client = MongoDBClient(uri=os.environ['MONGO_CONNECTION_STRING'])
client.connect()

# 저장소 초기화
content_repo: YoutubeContentRepository = YoutubeContentRepository(client)

# LLM
openai_llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)


# Content
contents = content_repo.find_all()
content: YouTubeContent = contents[1]

Connected to MongoDB


# 아웃 파서

In [77]:
from typing import List
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

# ✅ (핵심 키포인트 모델)
class YoutubeKeyPointModel(BaseModel):
    term: str = Field(..., description="핵심 용어")
    description: str = Field(..., description="핵심 용어에 대한 설명")

# ✅ (핵심 키포인트 컬렉션 모델)
class YoutubeKeyPointCollectionModel(BaseModel):
    key_points: List[YoutubeKeyPointModel] = Field(..., description="영상에서 추출된 핵심 키포인트 리스트")

# ✅ 아웃파서 정의
key_point_parser = PydanticOutputParser(pydantic_object=YoutubeKeyPointCollectionModel)

# 프롬프트 템플릿

In [78]:
from langchain.prompts import PromptTemplate

key_point_prompt_template = PromptTemplate(
    partial_variables={'output_format': key_point_parser.get_format_instructions()},
    input_variables=["title", "descrption", "summary"],
    template="""
다음은 유튜브 영상의 제목, 설명 타임라인 요약 문서입니다. 이 내용을 분석하여 핵심 용어(KeyPoint)와 설명을 생성하세요.
설명을 생성할 때 영상 위주가 아닌 최대한 전문적이고 구체적으로 추가 설명해주세요. 저는 영상 내용 뿐 아니라 그 용어에 대한 내용을 알고 싶습니다.

[제목]:
{title}

[설명]:
{description}

[타임라인 요약]:
{summary}


[가이드]:
- 필드는 `term`과 `description`을 포함하며, 결과는 여러 개의 키포인트 리스트로 구성됩니다.
- 반드시 아래 JSON 형식에 맞춰 반환하세요.
- 다른 텍스트는 포함하지 마세요.

[반환 형식]:
{output_format}
"""
)

# LLM

In [79]:
from langchain_openai import ChatOpenAI

openai_llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.environ['OPENAI_API_KEY'])

In [80]:
script_refine_chain = key_point_prompt_template | openai_llm | key_point_parser


summary: YoutubeTimelineSummary = content.timeline_summary

summary_texts = f"{summary.text}\n"

for section in summary.sections:
    summary_texts = summary_texts + '\n'
    for text in section.texts:
        summary_texts = summary_texts + f'- {text}\n'


print(key_point_prompt_template)

input_variables=['description', 'summary', 'title'] input_types={} partial_variables={'output_format': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"YoutubeKeyPointModel": {"properties": {"term": {"description": "핵심 용어", "title": "Term", "type": "string"}, "description": {"description": "핵심 용어에 대한 설명", "title": "Description", "type": "string"}}, "required": ["term", "description"], "title": "YoutubeKeyPointModel", "type": "object"}}, "properties": {"key_points": {"description": "영상에서 추출된 핵심 키포인트 리스트", "items": {"$ref": "#/$defs/YoutubeKeyPointModel"}, "title": "Key Poi

In [81]:
input_data = {
        "title": content.title,
        "description": content.description,
        "summary": summary_texts
    }

response: YoutubeKeyPointCollectionModel = script_refine_chain.invoke(input_data)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [82]:
for key_point in response.key_points:
    print(f'{key_point.term}: {key_point.description}')

C# 스크립트: 유니티의 스크립트 작성 언어인 C#을 사용하여 게임 로직을 구현합니다. C#은 강력한 객체 지향 프로그래밍 언어로, 유니티에서는 주로 게임 오브젝트의 행동이나 상호작용을 정의하기 위해 사용됩니다.
플레이어 이동: 게임 내에서 플레이어 캐릭터를 이동시키는 기능으로, 일반적으로 사용자 입력에 따라 캐릭터가 좌우, 상하로 반응하는 방식으로 구현됩니다. 이는 게임의 기본적인 조작 방식 중 하나로, 사용자 경험에 큰 영향을 미칩니다.
Input.GetAxis(): 유니티에서 사용자 입력을 감지하기 위해 사용하는 함수로, 'Horizontal'과 'Vertical' 축의 입력 값을 가져옵니다. 이를 통해 키보드, 게임패드와 같은 장치로 부터의 이동 방향을 감지할 수 있습니다.
RigidBody2D: 유니티에서 2D 물리 엔진에 기반한 물체의 물리적 성질을 다루기 위한 컴포넌트로, 물리 시뮬레이션을 통해 오브젝트에 힘을 적용하거나 충돌을 처리할 수 있게 합니다. 플레이어 이동 및 물리 상호작용에 필수적입니다.
FixedUpdate: 유니티의 스크립트에서 물리 연산을 위한 업데이트 메서드로, 매 프레임이 아닌 일정한 시간 간격으로 호출되어 물리 엔진 관련 코드 블록을 실행합니다. 이는 물리 이동을 구현할 때 일정한 자연스러운 움직임을 보장합니다.
AddForce: RigidBody2D를 통해 물리적인 힘을 객체에 직접적으로 추가하는 메서드로, 이를 이용해 객체의 속도를 변화시킬 수 있습니다. 물리 기반의 움직임을 보다 사실감 있게 구현할 수 있습니다.
Normalize: 벡터의 방향을 유지하면서 벡터의 크기를 1로 조정하는 과정으로, 이를 통해 방향을 유도할 수 있습니다. 게임에서 방향 벡터를 정규화함으로써 일관된 이동 속도를 유지할 수 있습니다.
FixedDeltaTime: 유니티에서 물리 시뮬레이션과 관련된 시간 간격을 정의하는 값으로, 물리 업데이트 간의 시간 간격을 일정하게 유지함으로써 정밀한 물리 연산을 가능하게 합니다. 이는 이동 속도를 일정하게 